<a href="https://colab.research.google.com/github/mugalan/working/blob/main/hierarchical_graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# References

* https://arxiv.org/pdf/2303.03293
* https://ris.utwente.nl/ws/files/52018562/Kienreich2012graph.pdf

# Python Class Definition

In [ ]:
import networkx as nx
import graphviz
from IPython.display import display, Image

class HierarchicalGraph:
    def __init__(self, nodes_data, edges_data):
        self.nodes_data = nodes_data
        self.edges_data = edges_data
        self._assign_group_colors()
        self.inner_graph = nx.MultiDiGraph()
        self.outer_graph = nx.DiGraph()
        self.create_hierarchical_graphs_iterative()


    def _assign_group_colors(self):
        """Assign consistent colors to groups."""
        default_colors = [
            'lightblue', 'lightgreen', 'lightyellow', 'lightpink', 'lightgray',
            'lightsalmon', 'lightcyan', 'wheat', 'plum', 'lightgoldenrod'
        ]
        groups = sorted(set(node['group'] for node in self.nodes_data))
        self.group_colors = {
            group: default_colors[i % len(default_colors)]
            for i, group in enumerate(groups)
        }



    def create_hierarchical_graphs_iterative(self):
        inner_graph = nx.MultiDiGraph()

        # --- Add nodes to inner graph ---
        for node_data in self.nodes_data:
            label = node_data['label']
            attributes = {k: v for k, v in node_data.items() if k != 'label'}
            inner_graph.add_node(label, **attributes)

        # --- Add edges to inner graph ---
        for edge_data in self.edges_data:
            start = edge_data['start']
            end = edge_data['end']
            attributes = {k: v for k, v in edge_data.items() if k not in ['start', 'end']}
            inner_graph.add_edge(start, end, **attributes)

        # --- Create outer MultiDiGraph ---
        outer_graph = nx.MultiDiGraph()

        # --- Add group nodes to outer graph ---
        group_nodes = set(d['group'] for d in self.nodes_data)
        outer_graph.add_nodes_from(group_nodes)

        # --- Add inter-group edges to outer graph (one-to-one mapping from inner) ---
        for u, v, data in inner_graph.edges(data=True):
            group_u = inner_graph.nodes[u].get('group')
            group_v = inner_graph.nodes[v].get('group')

            if group_u != group_v:
                # Preserve edge attributes (e.g., type, weight, color)
                outer_graph.add_edge(group_u, group_v, **data)

        # Assign back
        self.inner_graph = inner_graph
        self.outer_graph = outer_graph


    # --------------------------
    # Node Operations
    # --------------------------
    def add_node(self, node_data):
        self.nodes_data.append(node_data)
        self.create_hierarchical_graphs_iterative()

    def edit_node(self, label, new_data):
        for node in self.nodes_data:
            if node['label'] == label:
                node.update(new_data)
                break
        self.create_hierarchical_graphs_iterative()

    def delete_node(self, label):
        self.nodes_data = [node for node in self.nodes_data if node['label'] != label]
        self.edges_data = [edge for edge in self.edges_data if edge['start'] != label and edge['end'] != label]
        self.create_hierarchical_graphs_iterative()

    # --------------------------
    # Edge Operations
    # --------------------------
    def add_edges(self, edge_data):
        """Add one or more edges."""
        if isinstance(edge_data, list):
            self.edges_data.extend(edge_data)
        else:
            self.edges_data.append(edge_data)
        self.create_hierarchical_graphs_iterative()

    def edit_edges(self, start_end_pairs, new_data):
        """
        Edit one or more edges.
        `start_end_pairs`: tuple or list of tuples like [(start, end), ...]
        `new_data`: dict with new edge attributes
        """
        if isinstance(start_end_pairs, tuple):
            start_end_pairs = [start_end_pairs]

        for start, end in start_end_pairs:
            for edge in self.edges_data:
                if edge['start'] == start and edge['end'] == end:
                    edge.update(new_data)
                    break
        self.create_hierarchical_graphs_iterative()

    def delete_edges(self, start_end_pairs):
        """
        Delete one or more edges.
        `start_end_pairs`: tuple or list of tuples like [(start, end), ...]
        """
        if isinstance(start_end_pairs, tuple):
            start_end_pairs = [start_end_pairs]

        self.edges_data = [
            edge for edge in self.edges_data
            if (edge['start'], edge['end']) not in start_end_pairs
        ]
        self.create_hierarchical_graphs_iterative()


    # --------------------------
    # Visualization Methods
    # --------------------------
    def visualize_outer_graph(self, filename='outer_level_graph'):
        dot = graphviz.Digraph(comment='Outer Level Graph', engine='dot')

        # Add nodes with group-based fill color
        for node in self.outer_graph.nodes:
            fillcolor = self.group_colors.get(node, 'white')
            dot.node(str(node), style='filled', fillcolor=fillcolor)

        # Add directed edges with attributes
        for u, v, key, data in self.outer_graph.edges(keys=True, data=True):
            label = data.get('type', '')
            penwidth = str(data.get('weight', 1.0) * 2)
            color = data.get('color', 'black')
            dot.edge(str(u), str(v),
                    label=label,
                    penwidth=penwidth,
                    color=color)

        filepath = dot.render(filename, format='png', cleanup=True)
        print(f"Outer-level graph saved as {filepath}")
        try:
            display(Image(filename=filepath))
        except:
            print("Open the saved image to view the graph.")

    def visualize_inner_graph_with_clusters(self, filename="inner_level_graph"):
        dot_source = 'digraph G {\n'
        dot_source += '    rankdir="LR";\n'
        dot_source += '    node [shape=box, style="filled"];\n'

        # Group nodes by their cluster (group)
        clusters = {}
        node_attrs = {}
        for node, attrs in self.inner_graph.nodes(data=True):
            group = attrs.get('group', 'DefaultGroup')
            clusters.setdefault(group, []).append(node)
            node_attrs[node] = attrs

        for group, nodes in clusters.items():
            bg_color = self.group_colors.get(group, 'white')
            cluster_id = group.replace(" ", "_")

            dot_source += f'    subgraph cluster_{cluster_id} {{\n'
            dot_source += f'        label="{group}";\n'
            dot_source += f'        bgcolor="{bg_color}";\n'

            for node in nodes:
                # Fallback to group color if no individual color provided
                node_color = node_attrs[node].get('color', self.group_colors.get(group, 'white'))
                dot_source += f'        "{node}" [fillcolor="{node_color}"];\n'

            dot_source += '    }\n'

        # Add edges
        for u, v, key, data in self.inner_graph.edges(keys=True, data=True):
            label = data.get('type', '')
            penwidth = str(data.get('weight', 1.0) * 2)
            color = data.get('color', 'black')
            dot_source += f'    "{u}" -> "{v}" [label="{label}", id="{key}", penwidth={penwidth}, color="{color}"];\n'

        dot_source += '}\n'

        dot = graphviz.Source(dot_source, format='png')
        filepath = dot.render(filename, cleanup=True)
        print(f"Inner-level graph saved as {filepath}")
        try:
            display(Image(filename=filepath))
        except:
            print("Open the saved image to view the graph.")


In [ ]:
nodes = [
    {'label': 'A', 'group': 'Group A', 'type': 'system', 'color':'yellow'},
    {'label': 'B', 'group': 'Group A', 'type': 'user','color':'yellow'},
    {'label': 'C', 'group': 'Group B', 'type': 'system','color':'orange'},
    {'label': 'D', 'group': 'Group B', 'type': 'user','color':'orange'},
    {'label': 'E', 'group': 'Group C', 'type': 'system','color':'pink'},
]

edges = [
    {'start': 'A', 'end': 'B', 'type': 'type1', 'weight': 0.4, 'color':'blue'},
    {'start': 'A', 'end': 'C', 'type': 'type2', 'weight': 0.8},
    {'start': 'B', 'end': 'C', 'type': 'type1', 'weight': 0.5},
    {'start': 'C', 'end': 'E', 'type': 'type3', 'weight': 0.1},
    {'start': 'C', 'end': 'E', 'type': 'type1', 'weight': 0.9}, # Multi-edge here
    {'start': 'D', 'end': 'E', 'type': 'type2', 'weight': 0.3},
]

In [ ]:
hg = HierarchicalGraph(nodes, edges)
hg.visualize_inner_graph_with_clusters()
hg.visualize_outer_graph()

In [ ]:
hg.add_node({'label': 'F', 'group': 'Group A', 'type': 'user'})

In [ ]:
hg.add_edges(edge_data=[
    {'start': 'C', 'end': 'A', 'type': 'type5', 'weight': 0.4},
    {'start': 'F', 'end': 'E', 'type': 'type5', 'weight': 0.4}
])

In [ ]:
hg.visualize_inner_graph_with_clusters()